In [60]:
import torch
import numpy as np

In [61]:
batch = 4
seq_len = 2
dim = 2
a=torch.rand(batch,seq_len,dim)
b=torch.rand(batch,1,dim)
# b = b.repeat(1,seq_len,1)
# res = torch.concat([a,b],dim=-1)
res = a+b
print(a,b,res)
print(res.shape)

tensor([[[0.8554, 0.8157],
         [0.5281, 0.3419]],

        [[0.6458, 0.0837],
         [0.5861, 0.2356]],

        [[0.0338, 0.2975],
         [0.2715, 0.5983]],

        [[0.9611, 0.7021],
         [0.6845, 0.3471]]]) tensor([[[0.1939, 0.5738]],

        [[0.7665, 0.1420]],

        [[0.3159, 0.9313]],

        [[0.2528, 0.0822]]]) tensor([[[1.0494, 1.3895],
         [0.7220, 0.9157]],

        [[1.4123, 0.2257],
         [1.3526, 0.3776]],

        [[0.3497, 1.2288],
         [0.5873, 1.5296]],

        [[1.2139, 0.7843],
         [0.9373, 0.4293]]])
torch.Size([4, 2, 2])


In [62]:
output = torch.rand(4,5)
target = torch.randint(0,5,(4,1))
pred = output.topk(5, 1, True, True)[1].t()
expand_target = target.view(1, -1).expand_as(pred)
correct = pred.eq(expand_target)

In [63]:
output = torch.rand(4,5)
top1 = output.topk(1)[1]
print(top1[0])

tensor([1])


In [64]:
import random
res = random.sample(range(1000),10)
print(res)

[924, 264, 569, 494, 635, 560, 393, 756, 125, 600]


In [77]:
def sapmle_topk1_prototypes(logits:torch.Tensor, targets:torch.Tensor, topK:int):
    cls_num = logits.shape[1]
    batch = logits.shape[0]
    

    origin_target = []
    new_target = torch.zeros(batch,1).view(-1)
    for i, target in enumerate(targets):
        random_targets = random.sample(range(cls_num),topK+1)
        if target in random_targets:
            new_target[i] = random_targets.index(target)
        else:
            random_targets[0] = target.numpy()
        origin_target =  np.append(origin_target, random_targets)
    
    origin_target = origin_target.reshape(batch,-1)
    new_target = new_target.cuda()

    return new_target, origin_target

In [78]:
cls_num = 10
batch = 4
logits = torch.rand(batch,cls_num)
target = torch.randint(0,cls_num,(batch,1)).view(-1)
new_target, origin_target = sapmle_topk1_prototypes(logits,target,5)

print(logits.topk(5)[1])
print(target)
print(new_target)
print(origin_target)

tensor([[6, 4, 5, 3, 8],
        [1, 4, 0, 2, 7],
        [1, 9, 2, 7, 4],
        [4, 5, 9, 8, 1]])
tensor([4, 3, 8, 0])
tensor([0., 0., 0., 0.], device='cuda:0')
[[4. 5. 7. 8. 1. 6.]
 [3. 0. 7. 4. 8. 6.]
 [8. 9. 2. 1. 4. 6.]
 [0. 5. 7. 1. 3. 2.]]


# .numpy()

In [82]:
a = torch.tensor([1,2])
print(a)
b = a.numpy()
print(b)

tensor([1, 2])
[1 2]


# sample

In [96]:
random.seed(1)
for i in range(2):
    for j in range(2):
        res = random.sample(range(10),5)
        print(res)

[2, 1, 4, 0, 3]
[7, 9, 6, 8, 1]
[1, 7, 0, 6, 3]
[6, 0, 7, 2, 5]


# att rank

In [143]:
batch = 1
k = 2
y = abs(torch.rand(batch, k))
res = abs(torch.rand(batch, k))
print(y)
print(res)

tensor([[0.8761, 0.2455]])
tensor([[0.6298, 0.5212]])


In [147]:
def att_rank(y):
    exp_y = torch.exp(y)
    # print(f'exp_y:{exp_y}')
    sum = torch.sum(exp_y,dim=-1).unsqueeze(-1)
    # print(f'sum:{sum}')
    y = exp_y / sum
    return y

In [151]:
y = att_rank(y)
res = att_rank(res)
loss1 = y*torch.log(res)
loss2 = (1-y)*torch.log(1-res)
Loss = loss1+loss2
print(Loss.sum())

tensor(-1.3862)


In [ ]:
index = torch.arange(batch)
pos_score = sim_mat[index, labels] # [batch] ， 每行是每个样本与其对应类中心的距离

all_score = torch.sum(sim_mat,dim=-1) # [batch]， 每行是每个样本与所有类中心距离之和

loss = - torch.log(pos_score / all_score) # 每个样本都要与其类中心最接近